##### imports

In [84]:
from numpy import exp
from numpy import sqrt
from numpy import cos
from numpy import e
from numpy import pi
from typing import List
import random

#### Definindo váriaveis e função de avaliação

In [86]:
TAM_DA_POPULACAO = 10
POPULACAO = []
MUTACOES_POP = []
D_DE_FILHOS = 7 * TAM_DA_POPULACAO


In [87]:
A = 20
B = 0.2
C = 2 * pi
D = 30

In [88]:
def ackley_funct(x: List[float]):
  return - A * exp(-B * sqrt(sum(map(lambda n: n ** 2, x)) / D)) \
          - exp(sum(map(lambda n: cos(C * n), x)) / D) \
          + A + exp(1)

In [97]:
print(f'{ackley_funct([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]) : .15f}') 

 0.000000000000000


# Versão 1 - Estratégia Evolutiva

## Inicialização da população

In [91]:
def generate_individual():
  x = []
  for i in range(D):
    x.append(random.uniform(-15, 15))
    
  return x

In [96]:
def generate_population():
  global POPULACAO, TAM_DA_POPULACAO, MUTACOES_POP
  
  for _ in range(TAM_DA_POPULACAO):
    POPULACAO.append(generate_individual())
    MUTACOES_POP.append(random.normalvariate(0,1))

In [50]:
def reset_population() -> None:
    global POPULACAO

    POPULACAO = []

## Seleção de pais

In [71]:
def select_parents():
    global POPULACAO

    pai1, pai2 = random.sample(POPULACAO, 2)

    return pai1, pai2

## Recombinação

In [83]:
# Local discreto
def evolutive_crossover(pai1: List[float], pai2: List[float]):
    filho = []
    for i in range(D):
        filho += random.sample([pai1[i], pai2[i]], 1)

    return filho

In [82]:
# inicializar EE
reset_population()
generate_population()

filhos = []
for _ in range(QTD_DE_FILHOS):
    pai1, pai2 = select_parents()
    filhos.append(evolutive_crossover(pai1, pai2))

print(filhos[0])

30


## Mutação

In [ ]:
def regra_1_quinto(sigma: float, sucesso: float, c: float) -> float:
    if sucesso > 1 / 5:
        sigma = sigma / c
    elif sucesso < 1 / 5:
        sigma = sigma * c
    
    return sigma

# Versão 2 - Estratégia Genética